Парсинг

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

sparql.setQuery("""
SELECT ?item ?itemLabel ?ruArticle WHERE {
  ?item wdt:P31 wd:Q6256.
  OPTIONAL {
    ?ruArticle schema:about ?item;
               schema:isPartOf <https://ru.wikipedia.org/>.
  }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "ru". }
}
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result["itemLabel"]["value"], "→", result.get("ruArticle", {}).get("value"))


In [ ]:
def clean_text(text):
    # Убирает лишние пробелы и переносы строк
    return ' '.join(text.split())


In [ ]:
def is_inside_infobox(tag):
    # Проверяем, есть ли в цепочке родителей таблица с классом infobox
    for parent in tag.parents:
        if parent.name == "table" and "infobox" in parent.get("class", []):
            return True
    return False

def get_intro(url):
    try:
        response = requests.get(url)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')

        content_div = soup.find('div', id='mw-content-text')
        if not content_div:
            return "Ошибка: не найден блок с содержимым статьи."

        intro_paragraphs = []
        for elem in content_div.find_all(recursive=True):
            if elem.name == 'h2':
                break
            if elem.name == 'p':
                if is_inside_infobox(elem):
                    continue  # пропускаем абзацы из боковой таблицы
                text = elem.get_text(" ", strip=True)
                if text:
                    intro_paragraphs.append(text)

        raw_text = '\n\n'.join(intro_paragraphs)
        return clean_text(raw_text) if raw_text else "Введение не найдено."
    except Exception as e:
        return f"Ошибка: {e}"


In [ ]:
for result in results["results"]["bindings"]:
    label = result["itemLabel"]["value"]
    article_url = result.get("ruArticle", {}).get("value")
    if article_url:
        intro = get_intro(article_url)
        print(f"{label}")
        print(article_url)
        print(intro)
        print()  # пустая строка для читаемости

In [ ]:
def clean_text(text):
    # Удаляем невидимые символы
    text = re.sub(r'[\u200e\u200f\u202a\u202b\u202c\u2066\u2067\u2068\u2069]', '', text)

    # Удаляем сноски вида [1], [23]
    text = re.sub(r'\[\d+\]', '', text)

    # Удаляем транскрипции в скобках [æl baħˈreːn], [æl baħˈreːn] и пр.
    text = re.sub(r'\[[^\[\]]*?\ˈ?[^\[\]]*?\]', '', text)

    # Убираем "прослушать о файле"
    text = re.sub(r'прослушать о файле', '', text, flags=re.IGNORECASE)

    text = re.sub(r'о файле', '', text, flags=re.IGNORECASE)
    
    # Убираем сноски
    text = re.sub(r'\[\d+(,\s*\d+)*\]', '', text)
    
    # Убираем символы типа "▲ 0,467"
    text = re.sub(r'▲\s*\d+,\d+', '', text)
    
    #дополнительно убрать лишние пробелы
    text = re.sub(r'\s{2,}', ' ', text).strip()

    # Убираем пробелы перед знаками препинания
    text = re.sub(r'\s+([,.\-:;!?])', r'\1', text)

    # Убираем запятую перед закрывающей скобкой
    text = re.sub(r',\s*([\)\]\}])', r'\1', text)

    # Убираем пробелы после открывающих скобок
    text = re.sub(r'([\(\[\{])\s+', r'\1', text)

    # Убираем пробелы перед закрывающими скобками
    text = re.sub(r'\s+([\)\]\}])', r'\1', text)

    # Убираем пробелы внутри кавычек « ... »
    text = re.sub(r'«\s*([^»]*?)\s*»', r'«\1»', text)

    # Финальная очистка лишних пробелов
    text = re.sub(r'\s+', ' ', text)

    return text.strip()



for result in results["results"]["bindings"]:
    label = result["itemLabel"]["value"]
    article_url = result.get("ruArticle", {}).get("value")
    if article_url:
        intro = get_intro(article_url)
        intro_clean = clean_text(intro)
        print(f"{label}")
        print(article_url)
        print(intro_clean)
        print()  # пустая строка для читаемости между статьями


In [ ]:
with open('results.txt', 'w', encoding='utf-8') as f:
    for result in results["results"]["bindings"]:
        label = result["itemLabel"]["value"]
        article_url = result.get("ruArticle", {}).get("value")
        if article_url:
            intro = get_intro(article_url)
            intro_clean = clean_text(intro)
            f.write(f"{label}\n")
            f.write(f"{article_url}\n")
            f.write(f"{intro_clean}\n\n")  # пустая строка

У нас получился файл txt. Теперь мы его почистим

In [ ]:
# Чтение файла
with open('results.txt', 'r', encoding='utf-8') as f:
    content = f.read()

print(content)

In [ ]:
import unicodedata

def remove_accents(text):
    # Сначала заменяем "й" на временный маркер
    marker = '__Y_MARKER__'
    text = text.replace('й', marker).replace('Й', marker.upper())

    # Нормализуем текст в форму NFD
    normalized = unicodedata.normalize('NFD', text)

    # Удаляем все диакритики (категория "Mn"), кроме тех, что относятся к маркеру
    no_accents = ''.join(ch for ch in normalized if unicodedata.category(ch) != 'Mn')

    # Восстанавливаем букву "й"
    no_accents = no_accents.replace(marker, 'й').replace(marker.upper(), 'Й')

    return no_accents


cleaned_content = remove_accents(content)
print(cleaned_content)


In [ ]:
def remove_empty_parentheses(text):
    # Удаляем пустые круглые скобки (с пробелами внутри)
    text = re.sub(r'\(\s*\)', '', text)
    # Удаляем символ "➤"
    text = text.replace('➤', '')
    text = re.sub(r'МФА:;?', '', text)
    # Убираем пробел перед точкой
    text = re.sub(r'\s+\.', '.', text)
     # Убираем запятую с пробелом перед закрывающей скобкой
    text = re.sub(r',\s+\)', ')', text)
    # Убираем запятую, пробел и точку с запятой подряд
    text = re.sub(r',\s*;', '', text)
    # Убираем пробелы перед закрывающей круглой скобкой
    text = re.sub(r'\s+\)', ')', text)
    return text

clean_text = remove_empty_parentheses(cleaned_content)
print(clean_text)


In [ ]:
with open('wiki_corpus.txt', 'w', encoding='utf-8') as f:
    f.write(clean_text)

Лемматизация и создание json проходили в колабе, чтобы была возможность использовать mystem
https://colab.research.google.com/drive/1Po2-Hw4GsuzpzePs0SpwaJyCe0inFcb-?usp=sharing